In [210]:
import pandas as pd
import numpy as np
import os, sys

#the folder with all the morphometrix outputs
outputs = r'/Users/clarabird/Box Sync/OSU/GEMM/photogrammetry/body cond metric comparison/analysis/outputs/humpbacks'

#the folder where all the altitude csvs are
altcsvs = r'/Users/clarabird/Box Sync/OSU/GEMM/photogrammetry/body cond metric comparison/analysis/altitude/TripData'

#the folder where you want the csv to be saved
out = r'/Users/clarabird/Box Sync/OSU/GEMM/photogrammetry/body cond metric comparison/analysis'

## loop through and matchup with humpbacks

In [214]:
imgs = []
alts = []
csvs = []
df_a = pd.DataFrame(data = {})

#go through all the outputs folder and pull all csvs, assumes csv name is the image name
#this makes the list of measured images
for root,dirs,files in os.walk(outputs):
    imgs += [f.split(".")[0]+".JPG" for f in files if f.endswith('.csv')]

dfimg = pd.DataFrame(data={'Image_Orig':imgs})
dfimg['Image'] = [i.replace(" (1)","") for i in dfimg['Image_Orig']]
imgs1 = dfimg['Image'].tolist()
#loops through the altitude folders, makes list of those csvs
for root,dirs,files in os.walk(altcsvs):
    alts += [os.path.join(root,f) for f in files if f.endswith('.csv') and f.startswith('ImageM')]

#makes one big csv of all altitudes
for c in alts:
    df = pd.read_csv(c,sep = ',',usecols= ['Image','LaserAlt','BaroAlt']).dropna(subset = ['Image'])
    df_a = pd.concat([df_a,df])
    df_a['Image'] = [x.strip() for x in df_a['Image']]

#makes list of images that we have altitudes for and images that we don't have altitudes for
noalt = []
yesalt = []
for ix in imgs1:
    #i = ix + ".JPG"
    if ix in df_a['Image'].tolist():
        yesalt += [ix]
    else:
        #print(i,'False')
        noalt += [ix]

#print(yesalt)
#print(df_a['Image'].tolist())
#merge the altitude dataframe with the one with the original image name and the cleaned image name
#subset the altitude dataframe to just the images that we measured
df1 = df_a[df_a['Image'].isin(yesalt)]
#print(df1)
# #make the empty cell values contain a * for easier filtering
# df1['LaserAlt'] = df1['LaserAlt'].replace(np.nan,"*")
# df1['BaroAlt'] = df1['BaroAlt'].replace(np.nan,"*")

# #loop through laser and barometer, if laser is empty or 130, use the barometer
# talt = []
# for l,b in zip(df1['LaserAlt'],df1['BaroAlt']):
#     ll = str(l); bb = str(b)
#     if "*" not in ll:
#         talt += [l]
#     elif "*" in ll:
#         talt += [b]

# #add the altitude column
# df1['Altitude'] = talt
# df1['Pixel_Dimension'] = 0.00391667

#this part made a dataframe from the list with the focl and pixd values
#used the dates to match the focal lengths
indlist = os.path.join(out,"List_of_Individuals-MaRRS.xlsx")
dfi = pd.read_excel(indlist,sheet_name = 'humpbacks',usecols= ['Launch_Ht','Image','Cruise','Focal_Length'])
dfi['date'] = [x.split("_")[0] for x in dfi['Image']]
df1['date'] = [x.split("_")[0] for x in df1['Image']]

#make the empty cell values contain a * for easier filtering
df1['LaserAlt'] = df1['LaserAlt'].replace(np.nan,"*")
df1['BaroAlt'] = df1['BaroAlt'].replace(np.nan,"*")

#add the altitude column

df1['Pixel_Dimension'] = 0.00391667

#merge the dataframe with the focal length dataframe
df2 = df1.merge(dfi[['date','Focal_Length','Launch_Ht']],on='date').drop_duplicates(subset=['Image']).drop(['date'],axis=1)

# display(df1)
# display(dfi)
#loop through laser and barometer, if laser is empty or 130, use the barometer
talt = []
for l,b,ht in zip(df2['LaserAlt'],df2['BaroAlt'],df2['Launch_Ht']):
    ll = str(l); bb = str(b)
    if "*" not in ll:
        talt += [l]
    elif "*" in ll:
        bx = float(b) + float(ht)
        talt += [bx]

df2['Altitude'] = talt

df3 = df2.merge(dfimg,on='Image')
# display(df1)
# display(df2)
display(df3)

#save the dataframe as the saftey, name would be edited here.
oc = os.path.join(out,"humpbacks_safety.csv")
df3.to_csv(oc,sep = ',')

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,Image,LaserAlt,BaroAlt,Pixel_Dimension,Focal_Length,Launch_Ht,Altitude,Image_Orig
0,180307_L_F10_DSC00578.JPG,44.75,*,0.003917,50,1.995,44.75,180307_L_F10_DSC00578.JPG
1,180307_L_F10_DSC00605.JPG,54.04,*,0.003917,50,1.995,54.04,180307_L_F10_DSC00605.JPG
2,180307_L_F11_DSC00701.JPG,*,45.24,0.003917,50,1.995,47.235,180307_L_F11_DSC00701.JPG
3,180307_L_F2_DSC09572.JPG,51.21,48.15,0.003917,50,1.995,51.21,180307_L_F2_DSC09572.JPG
4,180307_L_F2_DSC09595.JPG,130*,47.8,0.003917,50,1.995,49.795,180307_L_F2_DSC09595.JPG
...,...,...,...,...,...,...,...,...
80,190301_L_F1_DSC06180.JPG,32.01,31.85,0.003917,35,2.230,32.01,190301_L_F1_DSC06180.JPG
81,181224_L_F1_DSC02206.JPG,68.79,68.3,0.003917,35,1.995,68.79,181224_L_F1_DSC02206.JPG
82,181224_L_F1_DSC02269.JPG,68.28,67.95,0.003917,35,1.995,68.28,181224_L_F1_DSC02269.JPG
83,181224_L_F1_DSC02311.JPG,69.58,68.9,0.003917,35,1.995,69.58,181224_L_F1_DSC02311.JPG


## loop through and merge with blues
because the List_of_Individuals-MaRRS doesn't have images per individual, for the blues I'm going to loop through the first row of the morphometrix outputs to pull ID per image.

In [233]:
#the folder with all the morphometrix outputs
outputs_blue = r'/Users/clarabird/Box Sync/OSU/GEMM/photogrammetry/body cond metric comparison/analysis/outputs/blues'

#the folder where all the altitude csvs are
altcsvs = r'/Users/clarabird/Box Sync/OSU/GEMM/photogrammetry/body cond metric comparison/analysis/altitude/TripData'

#the folder where you want the csv to be saved
out = r'/Users/clarabird/Box Sync/OSU/GEMM/photogrammetry/body cond metric comparison/analysis'

In [236]:
imgs = []
alts = []
df_a = pd.DataFrame(data = {})
ids = []
z = []
xs = []
csvs = []

keys = ['Image_Orig','Individual']
kDict = dict.fromkeys(keys)
y = pd.DataFrame(data={})

#go through all the outputs folder and pull all csvs, assumes csv name is the image name
#this makes the list of measured images
for root,dirs,files in os.walk(outputs_blue):
    imgs += [f.split(".")[0]+".JPG" for f in files if f.endswith('.csv')]
    csvs  += [os.path.join(root,c) for c in files if c.endswith('.csv')]


for cc in csvs:
    kDict['Image_Orig'] = os.path.split(cc)[1].split(".")[0]+".JPG"
    kDict['Individual'] = str(pd.read_csv(cc,nrows=1,header=None)[1].tolist()[0])
    yd = pd.DataFrame(data=kDict,index=[1])
    y = pd.concat([y,yd])

dfimg = pd.DataFrame(data={'Image_Orig':imgs})
dfimg['Image'] = [i.replace(" (1)","") for i in dfimg['Image_Orig']]
imgs1 = dfimg['Image'].tolist()
#loops through the altitude folders, makes list of those csvs
for root,dirs,files in os.walk(altcsvs):
    alts += [os.path.join(root,f) for f in files if f.endswith('.csv') and f.startswith('ImageM')]

#makes one big csv of all altitudes
for c in alts:
    df = pd.read_csv(c,sep = ',',usecols= ['Image','LaserAlt','BaroAlt']).dropna(subset = ['Image'])
    df_a = pd.concat([df_a,df])
    df_a['Image'] = [x.strip() for x in df_a['Image']]

#makes list of images that we have altitudes for and images that we don't have altitudes for
noalt = []
yesalt = []
for ix in imgs1:
    #i = ix + ".JPG"
    if ix in df_a['Image'].tolist():
        yesalt += [ix]
    else:
        #print(i,'False')
        noalt += [ix]

#print(yesalt)
#print(df_a['Image'].tolist())
#merge the altitude dataframe with the one with the original image name and the cleaned image name
#subset the altitude dataframe to just the images that we measured
df1 = df_a[df_a['Image'].isin(yesalt)]
#print(df1)


#this part made a dataframe from the list with the focl and pixd values
#used the dates to match the focal lengths
indlist = os.path.join(out,"List_of_Individuals-MaRRS.xlsx")
di = pd.read_excel(indlist,sheet_name = 'blues',usecols= ['Individual','Cruise','Focal_Length',"Launch_Ht"])

y['Individual'] = y['Individual'].str.replace("b","B")


dfi = di.merge(y,on='Individual',how='left')

dfii = dfi.merge(dfimg,on='Image_Orig',how='right')

#merge the dataframe with the focal length dataframe
df2 = df1.merge(dfii,on='Image',how = 'right')
#make the empty cell values contain a * for easier filtering
df2['LaserAlt'] = df2['LaserAlt'].replace(np.nan,"*")
df2['LaserAlt'] = df2['LaserAlt'].replace("nolaserfile","*")
df2['BaroAlt'] = df2['BaroAlt'].replace(np.nan,"*")
df2['Launch_Ht'] = [str(x).strip() for x in df2['Launch_Ht']]
df2['Launch_Ht'] = df2['Launch_Ht'].replace("",0)


#loop through laser and barometer, if laser is empty or 130, use the barometer
talt = []
for l,b,ht in zip(df2['LaserAlt'],df2['BaroAlt'],df2['Launch_Ht']):
    ll = str(l); bb = str(b)
    if "*" not in ll:
        talt += [l]
    elif "*" in ll:
        if b != "badbarofile":
            bx = float(b) + float(ht)
            talt += [bx]
        else:
            talt += [b]

#add the altitude column
df2['Altitude'] = talt
df2['Pixel_Dimension'] = 0.00391667
# display(df1)
# display(df2)
display(df2)

#save the dataframe as the saftey, name would be edited here.
oc = os.path.join(out,"blues_safety.csv")
df2.to_csv(oc,sep = ',')

,Image,LaserAlt,BaroAlt,Individual,Cruise,Focal_Length,Launch_Ht,Image_Orig,Altitude,Pixel_Dimension
0,170813_A_F1_DSC02690.JPG,72.010000000000005,badbarofile,Bm170813-44,IOS17,50,2.847,170813_A_F1_DSC02690.JPG,72.010000000000005,0.003917
1,170815_A_F2_DSC03745.JPG,76.14*,badbarofile,Bm170815-43,IOS17,50,2.9176,170815_A_F2_DSC03745.JPG,badbarofile,0.003917
2,170815_A_F3_DSC03871.JPG,52.270000000000003,badbarofile,Bm170815-21,IOS17,50,2.9176,170815_A_F3_DSC03871.JPG,52.270000000000003,0.003917
3,170815_A_F4_DSC03996.JPG,130*,80.70,Bm170815-28,IOS17,50,2.9176,170815_A_F4_DSC03996.JPG,83.6176,0.003917
4,170814_A_F2_DSC03182.JPG,*,84.03,Bm170814-51,IOS17,50,2.9176,170814_A_F2_DSC03182.JPG,86.9476,0.003917
5,170814_A_F3_DSC03207.JPG,*,71.9,Bm170814-40,IOS17,50,2.9176,170814_A_F3_DSC03207.JPG,74.8176,0.003917
6,170814_A_F3_DSC03322.JPG,*,71.57,Bm170814-50,IOS17,50,2.9176,170814_A_F3_DSC03322.JPG,74.4876,0.003917
7,170816_A_F1_DSC04045.JPG,130*,83.64,Bm170816-51,IOS17,50,2.9176,170816_A_F1_DSC04045.JPG,86.5576,0.003917
8,170816_A_F1_DSC04113.JPG,108.79*,84.61,Bm20170816-UnID1,IOS17,50,2.9176,170816_A_F1_DSC04113.JPG,87.5276,0.003917
9,170816_A_F2_DSC04211.JPG,130*,82.8,Bm170816-41,IOS17,50,2.9176,170816_A_F2_DSC04211.JPG,85.7176,0.003917
